https://076923.github.io/posts/Python-pytorch-13/  
https://sanghyu.tistory.com/24  
https://journal-home.s3.ap-northeast-2.amazonaws.com/site/2020kics/presentation/0565.pdf  

https://2bman.tistory.com/3

In [2]:
# Standard
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# from torch.utils.data import DataLoader
# from loader.custom_dataset import DS, collate_func

###### in_channels: input의 feature dimension
- out_channels: 내가 output으로 내고싶은 dimension
- kernel_size: time step을 얼마만큼 볼 것인가(=frame size = filter size)
- stride: kernel을 얼마만큼씩 이동하면서 적용할 것인가 (Default: 1)
- dilation: kernel 내부에서 얼마만큼 띄어서 kernel을 적용할 것인가 (Default: 1)
- padding: 한 쪽 방향으로 얼마만큼 padding할 것인가 (그 만큼 양방향으로 적용) (Default: 0)
- groups: kernel의 height를 조절 (Default: 1)
- bias: bias term을 둘 것인가 안둘 것인가 (Default: True)
- padding_mode: 'zeros', 'reflect', 'reflect', 'replicate', 'circular' (Default: 'zeros')  
  
- Input : [Batch_size, Feature_dimension, Time_step]
- Out : [Batch size, Feature dimension(Channel_dimension), kernel로 변경된 Time_step]

In [17]:
from glob import glob
from tqdm import tqdm
from torch.utils.data import DataLoader
from loader.custom_dataset import FinDataset, collate_func
from model.model import FinCNN

dataset = FinDataset(data_paths=sorted(glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))))

progress = tqdm(DataLoader(dataset, batch_size=2, shuffle=False, collate_fn=collate_func(True)), desc="Loading Data")

# for idx, inv_lambda, x, y in progress:
#     print(x)

Loading Data:   0%|          | 0/5992521 [00:00<?, ?it/s]

In [110]:
import torch
import torch.nn as nn
import numpy as np
conv = nn.Conv1d(in_channels=3, out_channels=2, kernel_size=1, padding=(1,), padding_mode="zeros")
conv(torch.tensor(np.ones((2, 3, 10)), dtype=torch.float32, requires_grad=False))

out = nn.Sigmoid()
y = out(conv(torch.tensor(np.ones((2, 3, 3)), dtype=torch.float32, requires_grad=False)))
torch.argmax(y, dim=2)

tensor([[1, 0],
        [1, 0]])

In [113]:
import model.model as arc

arc

<module 'model.model' from '/home/ubuntu/Fin/Finance/model/model.py'>

In [106]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output

tensor([[0.0931, 0.3990, 0.5079],
        [0.3536, 0.2547, 0.3917]])

In [108]:
torch.argmax(output, dim=1).reshape(-1, 1)

tensor([[2],
        [2]])

In [12]:
import scipy.stats as stats
import numpy as np

def featuring_x(x):
    if len(x) < 2:
        return np.column_stack([[0.], [0.], [0.], x["CCLD_DVSN"].values]).astype(np.float16)
    
    volume = stats.boxcox(x["CNTG_VOL"].values + 1e-9)
    amount = stats.boxcox((x["STCK_PRPR"].values * x["CNTG_VOL"].values) + 1e-9)
    trade_type = x["CCLD_DVSN"].values

    fx = np.column_stack([volume[0], amount[0], trade_type]) #stats.boxcox(t_diffs)[0]])
    inv_lambda = [volume[1], amount[1]]
    # rob_x = scaler.fit_transform(np.column_stack([volume, amount])) # t_diffs]))
    # pt = power_transform(np.column_stack([volume, amount, t_diffs]), method="")

    return fx, inv_lambda # np.column_stack([t_diffs, volume, amount, trade_type])

def calculate_prob_distribution(x, y):
    if any(((y["STCK_PRPR"] - x.iloc[-1]["STCK_PRPR"]) / x.iloc[-1]["STCK_PRPR"] >= 0.01).values) == True:
        return np.array([1], dtype=np.float16)
    else:
        return np.array([0], dtype=np.float16)


In [13]:
from torch.utils.data import Dataset
import pandas as pd
from datetime import datetime, time

class test(Dataset):
    def __init__(self, data_path, window_size=(3, 0), sliding_size=(0, 30), target_size=(1, 0)):
        self.data_path = data_path
        self.df = pd.read_parquet(self.data_path)
        
        self.window_size = pd.Timedelta(minutes=window_size[0], seconds=window_size[1])
        self.sliding_size = pd.Timedelta(minutes=sliding_size[0], seconds=sliding_size[1])
        self.target_size = pd.Timedelta(minutes=target_size[0], seconds=target_size[1])
        
        self.date_list = self.df['STCK_CNTG_HOUR'].dt.date.unique()
        
        # 다음 슬라이딩 시작 시간으로 업데이트
        self.start_time = self.df['STCK_CNTG_HOUR'].iloc[0]
        # 다음 슬라이딩 윈도우의 시작 인덱스 업데이트
        self.start_idx = self.df.index[self.df['STCK_CNTG_HOUR'] >= self.start_time].min()
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.start_time.time() < time(9, 0, 0):
            end_time = datetime.combine(self.start_time.date(), time(9, 0, 0))

            x = self.df[(self.df['STCK_CNTG_HOUR'] >= self.start_time) & (self.df['STCK_CNTG_HOUR'] < end_time)]
            y = self.df[(self.df['STCK_CNTG_HOUR'] >= end_time) & (self.df["STCK_CNTG_HOUR"] <= end_time + self.target_size)]

            # 다음 슬라이딩 시작 시간으로 업데이트
            self.start_time = end_time
            # 다음 슬라이딩 윈도우의 시작 인덱스 업데이트
            self.start_idx = self.df.index[self.df['STCK_CNTG_HOUR'] >= self.start_time].min()

        elif self.start_time >= datetime.combine(self.start_time.date(), time(15, 20, 0)) - self.target_size: # 
            try:
                end_time = datetime.combine(self.date_list[np.where(self.date_list > self.start_time.date())][0], time(9, 0, 0))
            except IndexError:
                self.idx += len(self.df[(self.df['STCK_CNTG_HOUR'] >= self.start_time)])

                self.reset()
                return self.__getitem__(idx)

            except StopIteration:
                raise StopIteration

            x = self.df[(self.df['STCK_CNTG_HOUR'] >= self.start_time) & (self.df['STCK_CNTG_HOUR'] < end_time)]
            y = self.df[(self.df['STCK_CNTG_HOUR'] >= end_time) & (self.df["STCK_CNTG_HOUR"] <= end_time + self.target_size)]

            # 다음 슬라이딩 시작 시간으로 업데이트
            self.start_time = end_time
            # 다음 슬라이딩 윈도우의 시작 인덱스 업데이트
            self.start_idx = self.df.index[self.df['STCK_CNTG_HOUR'] >= self.start_time].min()

        else:
            end_time = self.start_time + self.window_size
            x = self.df[(self.df['STCK_CNTG_HOUR'] >= self.start_time) & (self.df['STCK_CNTG_HOUR'] < end_time)]
            y = self.df[(self.df['STCK_CNTG_HOUR'] >= end_time) & (self.df['STCK_CNTG_HOUR'] <= end_time + self.target_size)]

            # 다음 슬라이딩 시작 시간으로 업데이트
            self.start_time += self.sliding_size
            # 다음 슬라이딩 윈도우의 시작 인덱스 업데이트
            self.start_idx = self.df.index[self.df['STCK_CNTG_HOUR'] >= self.start_time].min()

        if x.empty:
            return self.__getitem__(idx)
        
        return x, y

In [ ]:
class main_test(Dataset):
    def __init__():
        

In [ ]:
for x, y in test(sorted(glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet")))[0]):
    print(y)
    break

In [15]:
from base import BaseModel
import torch.nn as nn

class FinCNN(BaseModel):
    def __init__(self, num_classes):
        super(FinCNN, self).__init__()
        self.conv1 = nn.Sequential(
                nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1e-9),
                nn.ReLU(),
                nn.MaxPool1d(kernel_size=2, stride=2)
            )
        
        self.conv2 = nn.Sequential(
                nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1e-9),
                nn.AdaptiveAvgPool1d(1),  # Global Average Pooling
                nn.Flatten()
            )

        self.fc1 = nn.Linear(32, 128)  # GAP으로 인해 Linear의 input dimention == conv2의 out_channels
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [19]:
for batch_idx, (b_lambda, x, y) in progress:
    print(x)
    break

ValueError: too many values to unpack (expected 2)